In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
funding_rounds_df = pd.read_csv('/content/drive/MyDrive/Series A-nalysts/archive/funding_rounds.csv')
objects_df = pd.read_csv('/content/drive/MyDrive/Series A-nalysts/archive/objects.csv')

<ipython-input-2-35a9b52b15c3>:4: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  objects_df = pd.read_csv('/content/drive/MyDrive/Series A-nalysts/archive/objects.csv')


###Now we merge the objects_df which has all the companies' information and funding_rounds_df which has the information for funding rounds for each company into one dataframe

In [ ]:
full_df = pd.merge(objects_df, funding_rounds_df, left_on="id", right_on="object_id", how="inner")

### This is a simplification of our data and we can quickly see what to expect from our data.

In [ ]:
full_df_simplified = full_df[['id_x',  'founded_at', 'funding_rounds', 'funded_at', 'raised_amount_usd',  'is_last_round']]

###Here we fill na fields with the median and convert dates to datetime format. We also add columns such as company age at funding which is a numerical value. This allows our models to analyze the datetime information.

In [ ]:
# Sort the data
full_df_simplified = full_df_simplified.sort_values(by=['id_x', 'funded_at'])
full_df_simplified = full_df_simplified[full_df_simplified['raised_amount_usd'] < 40000000]

# Calculate cumulative sum of raised_amount_usd and current number of funding rounds
full_df_simplified['total_funding_until_this_point'] = full_df_simplified.groupby('id_x')['raised_amount_usd'].cumsum()
full_df_simplified['current_number_of_funding_rounds'] = full_df_simplified.groupby('id_x').cumcount() + 1

# Convert dates to datetime and create time features
full_df_simplified['founded_at'] = pd.to_datetime(full_df_simplified['founded_at'])
full_df_simplified['funded_at'] = pd.to_datetime(full_df_simplified['funded_at'])
full_df_simplified['company_age_at_funding'] = (full_df_simplified['funded_at'] - full_df_simplified['founded_at']).dt.days
full_df_simplified['time_since_last_round'] = full_df_simplified.groupby('id_x')['funded_at'].diff().dt.days.fillna(0)

# Shift target variable and drop rows where target is NaN
full_df_simplified['next_round_raised_amount_usd'] = full_df_simplified.groupby('id_x')['raised_amount_usd'].shift(-1)
full_df_simplified = full_df_simplified.dropna(subset=['next_round_raised_amount_usd'])

###Drop columns that have really low correlations or aren't used for prediction or that we repurposed into new columns

In [ ]:
full_df_simplified.drop(columns=['id_x', 'is_last_round', 'founded_at', 'funded_at'], inplace=True)

###Scale numerical columns

In [ ]:
# Columns to scale
numerical_cols = ['raised_amount_usd', 'total_funding_until_this_point', 'company_age_at_funding', 'time_since_last_round']

# Standardizing these columns
scaler = StandardScaler()
full_df_simplified[numerical_cols] = scaler.fit_transform(full_df_simplified[numerical_cols])

# Fill missing values with the median
full_df_simplified = full_df_simplified.fillna(full_df_simplified.median())


NameError: ignored

###Filtering of data on round raised amount

In [ ]:
full_df_simplified = full_df_simplified[full_df_simplified['current_number_of_funding_rounds'] < 4]

full_df_simplified_less = full_df_simplified[full_df_simplified['next_round_raised_amount_usd'] < 20000000]
full_df_simplified_more = full_df_simplified[full_df_simplified['next_round_raised_amount_usd'] >= 20000000]

full_df_simplified

KeyError: ignored

---



---

# END OF DATA PREPROCESSING

---



---

